In [3]:
import numpy as np
import pandas as pd

from scipy.stats import skew

from sklearn.linear_model import Ridge, Lasso
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [21]:
# Caminho dos dados.
DTRAIN_PATH = "data/train.csv"
DTEST_PATH = "data/test.csv"

# Funções auxiliares.
def reading_data(path):
    return pd.read_csv(path)

def rmse_cv(model, x_train, y_train, k_folds=10):
    return np.sqrt(-cross_val_score(model, x_train, y_train,
        scoring="neg_mean_squared_error", cv=k_folds))

def grid_search(model, params, train_x, train_y):
	# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
	# gridsearch = GridSearchCV(model, param_grid=params, scoring="neg_log_loss", n_jobs=-1, cv=kfold)

	gridsearch = GridSearchCV(model, param_grid=params, n_jobs=-1)
	gridresult = gridsearch.fit(train_x, train_y)

	return gridresult

def print_grid_result(name, grid_result):
	print(name + ":")
	# sumarize results
	print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
	means = grid_result.cv_results_['mean_test_score']
	stds = grid_result.cv_results_['std_test_score']
	params = grid_result.cv_results_['params']
	for mean, stdev, param in zip(means, stds, params):
		print("%f (%f) with: %r" % (mean, stdev, param))
	print("\n")


def grid_search_lasso(train_x, train_y):
	params = {'alpha':(1, 0.1, 0.001, 0.0005)}

	grid_result = grid_search(Lasso(), params, train_x, train_y)
	print_grid_result("Lasso", grid_result)

def grid_search_ridge(train_x, train_y):
	params = {'alpha':(1, 3, 5, 10, 15, 30, 50, 75)}

	grid_result = grid_search(Ridge(), params, train_x, train_y)
	print_grid_result("Ridge", grid_result)

In [117]:
# Funções interessantes
# all_data.head()
# all_data.tail()
# all_data.info()
# all_data.describe()

In [5]:
# Removing columns that the number of nans values is greater than bound=0.3 
def remove_nan_columns(df, bound=0.3):
    for column_name in df.columns:
        column = df[column_name]
        nan_percentage = column.isnull().sum()/column.size
        if(nan_percentage > bound):
            df = df.drop(columns=[column_name])
    return df

In [52]:
# function to normalise a column of values to lie between 0 and 1
def scale_minmax(col):
    return (col-col.min())/(col.max()-col.min())

# treating data with NaNs values
def fill_columns_with_NaN(df):
    # columns where NaN values have meaning, e.g. no pool, etc
    cols_fillna = ['PoolQC','MiscFeature','Alley','Fence','MasVnrType','FireplaceQu',
               'GarageQual','GarageCond','GarageFinish','GarageType',
               'BsmtExposure','BsmtCond','BsmtQual','BsmtFinType1','BsmtFinType2']
    
    # replacing NaN with None
    for col in cols_fillna:
        df[col].fillna('None', inplace=True)

    # GarageYrBlt nans: no garage, fill with property YearBuilt (zero isn't better?)
    df.loc[df.GarageYrBlt.isnull(), 'GarageYrBlt'] = df.loc[df.GarageYrBlt.isnull(), 'YearBuilt']
 
    # No mansonry veneer - fill with zeros.
    df.MasVnrArea.fillna(0, inplace=True)
        
    
    # No basement - fill areas/counts with 0
    df.BsmtFullBath.fillna(0,inplace=True)
    df.BsmtHalfBath.fillna(0,inplace=True)
    df.BsmtFinSF1.fillna(0,inplace=True)
    df.BsmtFinSF2.fillna(0,inplace=True)
    df.BsmtUnfSF.fillna(0,inplace=True)
    df.TotalBsmtSF.fillna(0,inplace=True)


    # No garage - fill areas/counts with 0
    df.GarageArea.fillna(0,inplace=True)
    df.GarageCars.fillna(0,inplace=True)
    
    
    # convert categoricals to dummies, exclude SalePrice from model
    df_frontage = pd.get_dummies(df)
    
    # normalise columns to 0-1
    for col in df_frontage.drop('LotFrontage',axis=1).columns:
        df_frontage[col] = scale_minmax(df_frontage[col])
    
    lf_train = df_frontage.dropna()
    lf_train_y = lf_train.LotFrontage
    lf_train_X = lf_train.drop('LotFrontage',axis=1)
    
    lr = Ridge()
    lr.fit(lf_train_X, lf_train_y)
    

    # fill na values using model predictions
    nan_frontage = df.LotFrontage.isnull()
    X = df_frontage[nan_frontage].drop('LotFrontage',axis=1)
    y = lr.predict(X)
    # fill nan values of lot frontage
    df.loc[nan_frontage,'LotFrontage'] = y
    
    # Remaining Nan values
    cols_with_na = df.isnull().sum()
    cols_with_na = cols_with_na[cols_with_na > 0]


    # fill remaining nans with mode in that column
    for col in cols_with_na.index:
        df[col].fillna(df[col].mode()[0], inplace=True)
    # fill remaining nans with mean in that column
    # df = df.fillna(all_data.mean())
    # fill remaining nans with median in that column
    # df = df.fillna(all_data.median())

        
    return df


In [19]:
# Removing rows that the number of nans values is greater than bound=0.1 (more than 10 values null)
def remove_nan_rows(df, bound=0.1):
    row_size = train_data.shape[1]
    for index, row in df.iterrows():
        rownan_percentage = row.isnull().sum()/row_size
        if (rownan_percentage >= bound ):
            df = df.drop(index)
    return df

In [6]:
# Removing redundance using drop_duplicates and correlation
def remove_redundance(df, bound=0.0005):
    df = df.drop_duplicates()

    corr = df.corr().abs()
    corr = corr[corr!= 1]
    print(corr.min() < bound)
    remove = np.empty(0)
    columns = corr.columns
    for index, row in corr.iterrows():
        for col in columns:
            if (row[col] < bound):
                remove = np.append(remove, col)
    print(df.shape)
    df = df.drop(columns=np.unique(remove))
    print(df.shape)

    return df

In [92]:
def basement_finish_types(df): 
    # create separate columns for area of each possible
    # basement finish type
    bsmt_fin_cols = ['BsmtGLQ','BsmtALQ','BsmtBLQ',
                     'BsmtRec','BsmtLwQ']

    for col in bsmt_fin_cols:
        # initialise as columns of zeros
        df[col+'SF'] = 0

    # fill remaining finish type columns
    for row in df.index:
        fin1 = df.loc[row, 'BsmtFinType1']

        if (fin1 != 'None') and (fin1 != 'Unf'):
            # add area (SF) to appropriate column
            df.loc[row, 'Bsmt' + fin1 + 'SF'] += df.loc[row,' BsmtFinSF1']

        fin2 = df_all.loc[row, 'BsmtFinType2']
        if (fin2 != 'None') and (fin2 != 'Unf'):
            df.loc[row, 'Bsmt' + fin2 + 'SF'] += df.loc[row, 'BsmtFinSF2']

    # remove initial BsmtFin columns
    df.drop(['BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2'], axis=1, inplace=True)

    # already have BsmtUnf column in dataset
    bsmt_fin_cols.append('BsmtUnf')

    # also create features representing the fraction of the basement that is each finish type
    for col in bsmt_fin_cols:
        df[col + 'Frac'] = df[col + 'SF'] / df['TotalBsmtSF']
        # replace any nans with zero (for properties without a basement)
        df[col + 'Frac'].fillna(0, inplace=True)
    
    return df

In [95]:
def floor_area(df):
    df['LowQualFinFrac'] = df['LowQualFinSF']/df['GrLivArea']
    df['1stFlrFrac'] = df['1stFlrSF']/df['GrLivArea']
    df['2ndFlrFrac'] = df['2ndFlrSF']/df['GrLivArea']

#     df['TotalAreaSF'] = df['GrLivArea']+df['TotalBsmtSF']+df['GarageArea']+df['EnclosedPorch']+df['ScreenPorch']
#     df['LivingAreaSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['BsmtGLQSF'] + df['BsmtALQSF'] + df['BsmtBLQSF']
#     df['StorageAreaSF'] = df['LowQualFinSF'] + df['BsmtRecSF'] + df['BsmtLwQSF'] + df['BsmtUnfSF'] + df['GarageArea']

    return df

In [102]:
def categorical_meaningful_ordering(df):
    # convert some categorical values to numeric scales

    #Excellent, Good, Typical, Fair, Poor, None: Convert to 0-5 scale
    cols_ExGd = ['ExterQual','ExterCond','BsmtQual','BsmtCond',
                 'HeatingQC','KitchenQual','FireplaceQu','GarageQual',
                'GarageCond','PoolQC']

    dict_ExGd = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'None':0}

    for col in cols_ExGd:
        df[col].replace(dict_ExGd, inplace=True)

    # Remaining columns
    df['BsmtExposure'].replace({'Gd':4,'Av':3,'Mn':2,'No':1,'None':0}, inplace=True)

    df['CentralAir'].replace({'Y':1,'N':0}, inplace=True)

    df['Functional'].replace({'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,'Maj2':2,'Sev':1,'Sal':0}, inplace=True)

    df['GarageFinish'].replace({'Fin':3,'RFn':2,'Unf':1,'None':0}, inplace=True)

    df['LotShape'].replace({'Reg':3,'IR1':2,'IR2':1,'IR3':0}, inplace=True)

    df['Utilities'].replace({'AllPub':3,'NoSewr':2,'NoSeWa':1,'ELO':0}, inplace=True)

    df['LandSlope'].replace({'Gtl':2,'Mod':1,'Sev':0}, inplace=True)
    
    return df

In [107]:
def dealing_with_zeros(df):
    # fraction of zeros in each column
    frac_zeros = ((df==0).sum()/len(df))

    # no. unique values in each column
    n_unique = df.nunique()

    # difference between frac. zeros and expected
    # frac. zeros if values evenly distributed between
    # classes
    xs_zeros = frac_zeros - 1/n_unique

    # create dataframe and display which columns may be problematic
    zero_cols = pd.DataFrame({'frac_zeros':frac_zeros,'n_unique':n_unique,'xs_zeros':xs_zeros})
    zero_cols = zero_cols[zero_cols.frac_zeros>0]
    zero_cols.sort_values(by='xs_zeros',ascending=False,inplace=True)
 
    #very few properties with Pool or 3SsnPorch
    #replace columns with binary indicator
    df['HasPool'] = (df['PoolQC']>0).astype(int)
    df['Has3SsnPorch'] = (df['3SsnPorch']>0).astype(int)
    df.drop(['PoolQC','PoolArea','3SsnPorch'],axis=1,inplace=True)

    # 'half' bathrooms - add half value to 'full' bathrooms
    df['BsmtFullBath'] = df['BsmtFullBath'] + 0.5*df['BsmtHalfBath']
    df['FullBath'] = df['FullBath'] + 0.5*df['HalfBath']
    df.drop(['BsmtHalfBath','HalfBath'],axis=1,inplace=True)

    # create additional dummy variable for
    # continuous variables with a lot of zeros
#     dummy_cols = ['LowQualFinSF','2ndFlrSF',
#                   'MiscVal','ScreenPorch','WoodDeckSF','OpenPorchSF',
#                   'EnclosedPorch','MasVnrArea','GarageArea','Fireplaces',
#                   'BsmtGLQSF','BsmtALQSF','BsmtBLQSF','BsmtRecSF',
#                   'BsmtLwQSF','BsmtUnfSF','TotalBsmtSF']

    dummy_cols = ['MiscVal','ScreenPorch','EnclosedPorch','MasVnrArea','GarageArea','Fireplaces']
    for col in dummy_cols:
        df['Has' + col] = (df[col] > 0).astype(int)
        
    return df

In [96]:
train_data = reading_data(DTRAIN_PATH)
test_data = reading_data(DTEST_PATH)

all_data = pd.concat((train_data.loc[:,'MSSubClass':'SaleCondition'],
                  test_data.loc[:,'MSSubClass':'SaleCondition']))

all_data = fill_columns_with_NaN(all_data)
# all_data = basement_finish_types(all_data)
all_data = floor_area(all_data)

In [117]:
def pre_processing_data(train_data, test_data):

    # Removendo as linhas com muitos dados faltantes do conjunto de treino.
    # train_data = remove_nan_rows(train_data)

    # Concatenando dados de treino e teste para facilitar as operações
    # de pré processamento.
    all_data = pd.concat((train_data.loc[:,'MSSubClass':'SaleCondition'],
                      test_data.loc[:,'MSSubClass':'SaleCondition']))

    # Preenchendo as colunas.
    all_data = fill_columns_with_NaN(all_data)
    
#     Não funcionou
#     all_data = basement_finish_types(all_data)

    # Aplicando Log nos preços de venda (valores em distribuição normal).
    train_data["SalePrice"] = np.log1p(train_data["SalePrice"])

#     numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

#     # Calcula o skewness
#     # For normally distributed data, the skewness should be about 0.
#     # A skewness value > 0 means that there is more weight in the left tail of the distribution.
#     skewed_feats = train_data[numeric_feats].apply(lambda x: skew(x.dropna()))

#     # Seleciona os valores com skewness > 0.75
#     skewed_feats = skewed_feats[skewed_feats > 0.75].index

#     # Aplicando log nos valores com skewness maior que 0.75
#     all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

    
    


   

    all_data = floor_area(all_data)
    all_data = categorical_meaningful_ordering(all_data)
    all_data = dealing_with_zeros(all_data)

        # extract names of numeric columns
    dtypes = all_data.dtypes
    cols_numeric = dtypes[dtypes != object].index.tolist()

    # MSubClass should be treated as categorical
    cols_numeric.remove('MSSubClass')
    
    # choose any numeric column with less than 13 values to be
    # "discrete". 13 chosen to include months of the year.
    # other columns "continuous"
    col_nunique = dict()

    for col in cols_numeric:
        col_nunique[col] = all_data[col].nunique()

    col_nunique = pd.Series(col_nunique)

    cols_discrete = col_nunique[col_nunique<13].index.tolist()
    cols_continuous = col_nunique[col_nunique>=13].index.tolist()

    # normalise numeric columns
    scale_cols = [col for col in cols_numeric if col!='SalePrice']

    all_data[scale_cols] = all_data[scale_cols].apply(scale_minmax, axis=0)
    
    
    # Converte dados categorigos em dummy indicators
    all_data = pd.get_dummies(all_data)

    # Preenchendo os valores em branco com a média.
    # all_data = all_data.fillna(all_data.mean())
    # Preenchendo os valores em branco com a mediana.
    # all_data = all_data.fillna(all_data.median())
    # Preenchendo os valores em branco com a moda.
    # all_data = all_data.fillna(all_data.mode().T)


    # Dados para treinamento e teste após pré processamento.
    x_train = all_data[:train_data.shape[0]]
    x_test = all_data[train_data.shape[0]:]
    y_train = train_data.SalePrice

    return (x_train, y_train, x_test)


train_data = reading_data(DTRAIN_PATH)
test_data = reading_data(DTEST_PATH)

train_x, train_y, test_x = pre_processing_data(train_data, test_data)


print('Lasso(alpha=0.0005):')
print(rmse_cv(Lasso(alpha=0.0005), train_x, train_y).mean())
# print(rmse_cv(Ridge(alpha=5), train_x, train_y).mean())
grid_search_lasso(train_x, train_y)
# grid_search_ridge(train_x, train_y)
print()
# Modelo XBG
# print('XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1):')
# print(rmse_cv(XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1), train_x, train_y).mean())


Lasso(alpha=0.0005):
0.13699331382727425
Lasso:
Best: 0.885809 using {'alpha': 0.0005}
0.000637 (0.000787) with: {'alpha': 1}
0.041406 (0.014399) with: {'alpha': 0.1}
0.881784 (0.015851) with: {'alpha': 0.001}
0.885809 (0.022454) with: {'alpha': 0.0005}





In [104]:
train_data = reading_data(DTRAIN_PATH)
test_data = reading_data(DTEST_PATH)

train_x, train_y, test_x = pre_processing_data(train_data, test_data)


print('Lasso(alpha=0.0005):')
print(rmse_cv(Lasso(alpha=0.0005), train_x, train_y).mean())
# print(rmse_cv(Ridge(alpha=10), train_x, train_y).mean())
grid_search_lasso(train_x, train_y)
grid_search_ridge(train_x, train_y)
print()
# Modelo XBG
print('XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1):')
print(rmse_cv(XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1), train_x, train_y).mean())


Lasso(alpha=0.0005):
0.12162855645714306
Lasso:
Best: 0.901133 using {'alpha': 0.0005}
0.545850 (0.032529) with: {'alpha': 1}
0.734174 (0.027527) with: {'alpha': 0.1}
0.897713 (0.017699) with: {'alpha': 0.001}
0.901133 (0.017624) with: {'alpha': 0.0005}


Ridge:
Best: 0.895923 using {'alpha': 5}
0.894325 (0.016889) with: {'alpha': 1}
0.895842 (0.016387) with: {'alpha': 3}
0.895923 (0.016170) with: {'alpha': 5}
0.895253 (0.015935) with: {'alpha': 10}
0.894283 (0.015782) with: {'alpha': 15}
0.891200 (0.015413) with: {'alpha': 30}
0.887570 (0.015049) with: {'alpha': 50}
0.883860 (0.014743) with: {'alpha': 75}



XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1):
0.12184203429783551


In [38]:
df = train_data
nans_cols = {}

for column_name in df.columns:
    column = df[column_name]
    nans = column.isnull().sum()
    if(nans > 0):
        nans_cols[column_name] = nans

# [print(v) for v in nans_cols]
import operator
sorted(nans_cols.items(),  key=operator.itemgetter(1))



[('Electrical', 1),
 ('MasVnrType', 8),
 ('MasVnrArea', 8),
 ('BsmtQual', 37),
 ('BsmtCond', 37),
 ('BsmtFinType1', 37),
 ('BsmtExposure', 38),
 ('BsmtFinType2', 38),
 ('GarageType', 81),
 ('GarageYrBlt', 81),
 ('GarageFinish', 81),
 ('GarageQual', 81),
 ('GarageCond', 81),
 ('LotFrontage', 259),
 ('FireplaceQu', 690),
 ('Fence', 1179),
 ('Alley', 1369),
 ('MiscFeature', 1406),
 ('PoolQC', 1453)]

# Results
## Default

Lasso(alpha=0.0005):
0.1211865670995667

XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1):
0.12249088341435015

## Removing nans values per col (upper_bound = 0.3)

Lasso(alpha=0.0005):
0.12082472393696742

XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1):
0.12304465867881136

## Removing nans values per row and cols
Lasso(alpha=0.0005):
0.11805243912299515

## Using Median
Lasso(alpha=0.0005):
0.1180499078769596

XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1):
0.116734983343264

## Using Mode
